In [1]:
import autograd.numpy as np
import pyipopt
from autograd import grad, hessian, jacobian

In [2]:
import matplotlib.pyplot as plt
from counterfactual_functions import *
from helper import *

username='boraozaltun'

data_dict = load_obj('/Users/'+username+'/Dropbox (MIT)/Data/Trade/general_equilibrium_gravity/final_data/data_20200219.pickle')

reg_2_num = load_obj('/Users/'+username+'/Dropbox (MIT)/Data/Trade/general_equilibrium_gravity/GTAP/mapping/reg_2_num.pickle')
comm_2_num = load_obj('/Users/'+username+'/Dropbox (MIT)/Data/Trade/general_equilibrium_gravity/GTAP/mapping/comm_2_num.pickle')

n = data_dict['n']
g = data_dict['g']
k = data_dict['k']

data_dict['R_hat'] = np.ones((n, g))
X_0 = np.ones(n*g + n)*1.0

eval_F = lambda x: reduced_counterfactual(x, data_dict)
eval_f = lambda x: np.linalg.norm(reduced_counterfactual(x, data_dict))


eval_grad_f = grad(eval_f)



In [3]:
nvar = X_0.shape[0]
x_L = np.zeros((nvar), dtype=float)
x_U = np.ones((nvar), dtype=float) * 100.0

ncon = 0


g_L = np.array([], dtype=float)
g_U = np.array([], dtype=float)



In [4]:
def eval_g(X, user_data=None):
    """
    Evaluate the constraint functions.
    """
    return np.array([], dtype=float)

nnzj = 0

def eval_jac_g(X, flag, user_data=None):
    """
    Evaluate the sparse Jacobian of constraint functions g.
    @param X: parameter values
    @param flag: this asks for the sparsity structure
    """
    print('eval_jac_g')
    print(X)
    print(flag)
    print(user_data)
    print()
    #XXX
    if flag:
        rows = np.array([], dtype=int)
        cols = np.array([], dtype=int)
        return (rows, cols)
    else:
        return np.array([], dtype=float)

nnzh = int((nvar*(nvar+1))/2)

def apply_new(x):
    return True

In [5]:
nlp = pyipopt.create(
            nvar,
            x_L,
            x_U,
            ncon,
            g_L,
            g_U,
            nnzj,
            nnzh,
            eval_f,
            eval_grad_f,
            eval_g,
            eval_jac_g)

nlp.num_option('tol', 1e-5)

True

In [6]:
results = nlp.solve(X_0)

eval_jac_g
[0.000e+000 1.136e-321 0.000e+000 ... 1.000e+002 1.000e+002 1.000e+002]
True
None



/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/autograd/numpy/numpy_vjps.py:444: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return lambda g: g[idxs]


In [8]:
x, zl, zu, constraint_multipliers, obj, status = results

In [9]:
nlp.close()

True

In [10]:
## If this is still not the right result, I think I might be doing something wrong in setting IPOPT up.

## Because this makes no sense. i am giving it a point where it is the minimum, but somehow it is still not
## converginng? Try with scipy and see what happens!

In [14]:
x

array([27.42525958, 27.42524017, 27.42528234, ..., 27.42543462,
       27.42521803, 27.42524074])

In [15]:
zu

array([9.06744335e-06, 9.06744092e-06, 9.06744619e-06, ...,
       9.06746522e-06, 9.06743816e-06, 9.06744099e-06])

In [16]:
constraint_multipliers

array([], dtype=float64)

In [17]:
obj

0.009584056029704937

In [18]:
status

-1

In [19]:
from scipy.optimize import root

In [20]:
sol = root(eval_F, X_0)

In [21]:
sol

    fjac: array([[-4.93850158e-01, -6.56199386e-05, -7.53603121e-05, ...,
        -1.13815145e-07,  3.67145631e-07,  8.07720387e-08],
       [-2.49571062e-03, -3.19132753e-01, -8.15480852e-03, ...,
         1.94599338e-06,  3.02232377e-06,  4.81961104e-06],
       [-4.21877720e-02, -9.66115303e-02, -8.76361365e-01, ...,
         3.49266467e-05,  3.94107389e-05,  4.29081300e-05],
       ...,
       [-5.28493367e-07, -5.28535853e-05,  3.42826873e-06, ...,
        -9.52888760e-01,  2.63429077e-01,  8.97408437e-02],
       [ 1.32943436e-06,  4.42217437e-06, -2.30103696e-05, ...,
        -6.67626764e-02, -5.06676745e-01,  8.51867714e-01],
       [-1.88164368e-06, -1.63125288e-04, -2.89970282e-05, ...,
        -2.24899219e-02, -2.07490480e-04, -4.02275486e-05]])
     fun: array([ 0.00000000e+00,  2.22044605e-16,  4.44089210e-16, ...,
        0.00000000e+00, -2.22044605e-16, -4.44089210e-16])
 message: 'The solution converged.'
    nfev: 3754
     qtf: array([-4.91104287e-16, -3.01364848e-16,

In [23]:
sol.x

array([1., 1., 1., ..., 1., 1., 1.])